## Import Dependencies

In [1]:
#Importing the dependencies
import pandas as pd
from pathlib import Path 
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression 
import psycopg2
import AWS_Password

In [2]:
#What kind of model
# Scikit-Learn Model Selection 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


## Connect to Databases and Clean & Pre-Process the Data (Happiness, Freedom, Life Expectancy)

In [3]:
#Connect Python to Postgres
import psycopg2
engine = psycopg2.connect(
    database="postgres",
    user="postgres",
    password=AWS_Password.password,
    host="happiness.c45odizugug2.us-east-2.rds.amazonaws.com",
    port='5432'
)
cursor = engine.cursor()
try:
    cursor.execute("""Select * From "World_Happiness_2015_2016_Report";""")
except (Exception, psycopg2.DatabaseError) as error:
    print("Error: %s" % error)
    cursor.close()
    
# Naturally we get a list of tuples
tuples = cursor.fetchall()
colnames = [desc[0] for desc in cursor.description]
cursor.close()
# We just need to turn it into a pandas dataframe
Happiness_Report_2015_2016_df = pd.DataFrame(tuples, columns=colnames)

In [4]:
# View Happiness_Report_2015_2016_df
Happiness_Report_2015_2016_df

,index,country,region,happiness_rank,happiness_score,economy_gdp_per_capita,family,health_life_expectancy,freedom,trust_government_corruption,generosity,dystopia_residual,year
0,1,Switzerland,Western Europe,1,7.587,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738,2015
1,2,Iceland,Western Europe,2,7.561,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201,2015
2,3,Denmark,Western Europe,3,7.527,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204,2015
3,4,Norway,Western Europe,4,7.522,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531,2015
4,5,Canada,North America,5,7.427,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,311,Benin,Sub-Saharan Africa,153,3.484,0.39499,0.10419,0.21028,0.39747,0.06681,0.20180,2.10812,2016
311,312,Afghanistan,Southern Asia,154,3.360,0.38227,0.11037,0.17344,0.16430,0.07112,0.31268,2.14558,2016
312,313,Togo,Sub-Saharan Africa,155,3.303,0.28123,0.00000,0.24811,0.34678,0.11587,0.17517,2.13540,2016
313,314,Syria,Middle East and Northern Africa,156,3.069,0.74719,0.14866,0.62994,0.06912,0.17233,0.48397,0.81789,2016


In [5]:
# Drop Index Column from Happiness dataframe
Happiness_Report_2015_2016_df = Happiness_Report_2015_2016_df.drop("index", axis=1)
Happiness_Report_2015_2016_df

,country,region,happiness_rank,happiness_score,economy_gdp_per_capita,family,health_life_expectancy,freedom,trust_government_corruption,generosity,dystopia_residual,year
0,Switzerland,Western Europe,1,7.587,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738,2015
1,Iceland,Western Europe,2,7.561,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201,2015
2,Denmark,Western Europe,3,7.527,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204,2015
3,Norway,Western Europe,4,7.522,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531,2015
4,Canada,North America,5,7.427,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176,2015
...,...,...,...,...,...,...,...,...,...,...,...,...
310,Benin,Sub-Saharan Africa,153,3.484,0.39499,0.10419,0.21028,0.39747,0.06681,0.20180,2.10812,2016
311,Afghanistan,Southern Asia,154,3.360,0.38227,0.11037,0.17344,0.16430,0.07112,0.31268,2.14558,2016
312,Togo,Sub-Saharan Africa,155,3.303,0.28123,0.00000,0.24811,0.34678,0.11587,0.17517,2.13540,2016
313,Syria,Middle East and Northern Africa,156,3.069,0.74719,0.14866,0.62994,0.06912,0.17233,0.48397,0.81789,2016


In [6]:
# Remove NaNs from Happiness dataframe
Happiness_Report_2015_2016_df=Happiness_Report_2015_2016_df.dropna()
Happiness_Report_2015_2016_df

,country,region,happiness_rank,happiness_score,economy_gdp_per_capita,family,health_life_expectancy,freedom,trust_government_corruption,generosity,dystopia_residual,year
0,Switzerland,Western Europe,1,7.587,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738,2015
1,Iceland,Western Europe,2,7.561,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201,2015
2,Denmark,Western Europe,3,7.527,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204,2015
3,Norway,Western Europe,4,7.522,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531,2015
4,Canada,North America,5,7.427,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176,2015
...,...,...,...,...,...,...,...,...,...,...,...,...
310,Benin,Sub-Saharan Africa,153,3.484,0.39499,0.10419,0.21028,0.39747,0.06681,0.20180,2.10812,2016
311,Afghanistan,Southern Asia,154,3.360,0.38227,0.11037,0.17344,0.16430,0.07112,0.31268,2.14558,2016
312,Togo,Sub-Saharan Africa,155,3.303,0.28123,0.00000,0.24811,0.34678,0.11587,0.17517,2.13540,2016
313,Syria,Middle East and Northern Africa,156,3.069,0.74719,0.14866,0.62994,0.06912,0.17233,0.48397,0.81789,2016


In [7]:
# Remove Duplicates from Happiness dataframe
Happiness_Report_2015_2016_df=Happiness_Report_2015_2016_df.drop_duplicates()
Happiness_Report_2015_2016_df

,country,region,happiness_rank,happiness_score,economy_gdp_per_capita,family,health_life_expectancy,freedom,trust_government_corruption,generosity,dystopia_residual,year
0,Switzerland,Western Europe,1,7.587,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738,2015
1,Iceland,Western Europe,2,7.561,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201,2015
2,Denmark,Western Europe,3,7.527,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204,2015
3,Norway,Western Europe,4,7.522,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531,2015
4,Canada,North America,5,7.427,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176,2015
...,...,...,...,...,...,...,...,...,...,...,...,...
310,Benin,Sub-Saharan Africa,153,3.484,0.39499,0.10419,0.21028,0.39747,0.06681,0.20180,2.10812,2016
311,Afghanistan,Southern Asia,154,3.360,0.38227,0.11037,0.17344,0.16430,0.07112,0.31268,2.14558,2016
312,Togo,Sub-Saharan Africa,155,3.303,0.28123,0.00000,0.24811,0.34678,0.11587,0.17517,2.13540,2016
313,Syria,Middle East and Northern Africa,156,3.069,0.74719,0.14866,0.62994,0.06912,0.17233,0.48397,0.81789,2016


In [8]:
#Connect Python to Postgres
import psycopg2
engine = psycopg2.connect(
    database="postgres",
    user="postgres",
    password=AWS_Password.password,
    host="happiness.c45odizugug2.us-east-2.rds.amazonaws.com",
    port='5432'
)
cursor = engine.cursor()
try:
    cursor.execute("""SELECT*FROM public."Human_Freedom_Index";""")
except (Exception, psycopg2.DatabaseError) as error:
    print("Error: %s" % error)
    cursor.close()
    
# Naturally we get a list of tuples
tuples = cursor.fetchall()
colnames = [desc[0] for desc in cursor.description]
cursor.close()
# We just need to turn it into a pandas dataframe
Human_Freedom_Index_df = pd.DataFrame(tuples, columns=colnames)

In [9]:
Human_Freedom_Index_df

,id,year,iso_code,countries,region,pf_rol_procedural,pf_rol_civil,pf_rol_criminal,pf_rol,pf_ss_homicide,...,ef_regulation_business_bribes,ef_regulation_business_licensing,ef_regulation_business_compliance,ef_regulation_business,ef_regulation,ef_score,ef_rank,hf_score,hf_rank,hf_quartile
0,1,2016,ALB,Albania,Eastern Europe,6.661502941,4.547243777,4.666508223,5.291751647,8.920429431,...,4.050195972,7.324582113,7.074365792,6.705863193,6.906901139,7.54,34.0,7.568140288,48.0,2.0
1,2,2016,DZA,Algeria,Middle East & North Africa,None,None,None,3.819566026,9.456253595,...,3.765515288,8.523503188,7.029528486,5.676955998,5.268992397,4.99,159.0,5.135886055,155.0,4.0
2,3,2016,AGO,Angola,Sub-Saharan Africa,None,None,None,3.451813885,8.060260239,...,1.945540493,8.096775933,6.782923304,4.930271338,5.518500441,5.17,155.0,5.640662182,142.0,4.0
3,4,2016,ARG,Argentina,Latin America & the Caribbean,7.098483117,5.791960457,4.343929507,5.744791027,7.622974220,...,3.260043621,5.253410644,6.508294805,5.535830648,5.369018774,4.84,160.0,6.469847841,107.0,3.0
4,5,2016,ARM,Armenia,Caucasus & Central Asia,None,None,None,5.003205353,8.808749692,...,4.575152278,9.319612283,6.491480816,6.797530175,7.378068688,7.57,29.0,7.241401801,57.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1453,1454,2008,VEN,Venezuela,Latin America & the Caribbean,3.000000000,3.781687785,2.369238962,3.100000000,0E-9,...,2.784694868,4.578317784,0.315141932,2.864160256,4.666935504,4.22,140.0,5.257879328,136.0,4.0
1454,1455,2008,VNM,Vietnam,South Asia,6.666666667,4.349101094,5.694846752,5.600000000,9.496239098,...,3.626592772,7.306073836,0E-9,4.574682645,6.307577197,6.19,103.0,6.420206308,106.0,3.0
1455,1456,2008,YEM,"Yemen, Rep.",Middle East & North Africa,None,None,None,None,None,...,None,None,None,None,None,None,NaN,None,NaN,NaN
1456,1457,2008,ZMB,Zambia,Sub-Saharan Africa,4.800000000,4.578003043,3.688651997,4.400000000,7.878084164,...,4.585391124,5.124435917,8.520368906,6.681524113,6.722174633,7.15,57.0,6.647724406,95.0,3.0


In [10]:
# Select columns to keep from the original Human_Freedom_Index csv
new_human_freedom_index_df = Human_Freedom_Index_df[["year", "countries", "pf_rol", "pf_ss", "pf_movement", "pf_religion", "pf_association", "pf_expression", "pf_identity", "pf_score", "ef_government", "ef_legal", "ef_money", "ef_trade", "ef_regulation", "ef_score", "hf_score"]]
new_human_freedom_index_df

,year,countries,pf_rol,pf_ss,pf_movement,pf_religion,pf_association,pf_expression,pf_identity,pf_score,ef_government,ef_legal,ef_money,ef_trade,ef_regulation,ef_score,hf_score
0,2016,Albania,5.291751647,8.806809810,6.666666667,9.192592593,10.000000000,8.607142857,6.250000000,7.596280576,7.935563809,5.071813726,9.553657390,8.214899510,6.906901139,7.54,7.568140288
1,2016,Algeria,3.819566026,8.043882199,5.000000000,4.944814815,5.000000000,7.380952381,0.833333333,5.281772109,3.616782254,4.690742835,7.253893581,4.127024824,5.268992397,4.99,5.135886055
2,2016,Angola,3.451813885,8.297864524,5.000000000,8.786666667,4.000000000,6.452380952,7.500000000,6.111324364,6.496684826,2.963635116,5.606605364,5.264754044,5.518500441,5.17,5.640662182
3,2016,Argentina,5.744791027,9.040383237,10.000000000,7.795925926,7.500000000,8.738095238,10.000000000,8.099695682,5.346056135,3.904030401,5.614335974,3.982546906,5.369018774,4.84,6.469847841
4,2016,Armenia,5.003205353,8.830548737,6.666666667,6.222222222,6.500000000,7.154761905,8.000000000,6.912803602,7.003309536,5.809214965,9.521939951,8.143241004,7.378068688,7.57,7.241401801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1453,2008,Venezuela,3.100000000,6.665877012,8.333333333,7.848148148,5.583333333,6.778079218,10.000000000,6.295758656,4.842725196,2.705464235,5.296955522,3.611472215,4.666935504,4.22,5.257879328
1454,2008,Vietnam,5.600000000,9.498746366,5.000000000,6.409259259,2.750000000,4.598000988,10.000000000,6.650412616,7.400000000,6.073716608,5.253293702,5.906879173,6.307577197,6.19,6.420206308
1455,2008,"Yemen, Rep.",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1456,2008,Zambia,4.400000000,8.181583610,8.333333333,7.714814815,4.250000000,7.202380952,2.500000000,6.145448813,7.867647059,5.810978462,8.367633930,6.983402160,6.722174633,7.15,6.647724406


In [11]:
#print(Human_Freedom_Index_df.isnull().sum().)
print(pd.DataFrame(new_human_freedom_index_df.isnull().sum()))

                  0
year              0
countries         0
pf_rol           80
pf_ss            80
pf_movement      80
pf_religion      90
pf_association  329
pf_expression    80
pf_identity      80
pf_score         80
ef_government    80
ef_legal         80
ef_money         82
ef_trade         81
ef_regulation    80
ef_score         80
hf_score         80


In [12]:
# Remove NaNs from Human Freedom Index dataframe
new_human_freedom_index_df= (new_human_freedom_index_df.dropna())
new_human_freedom_index_df


,year,countries,pf_rol,pf_ss,pf_movement,pf_religion,pf_association,pf_expression,pf_identity,pf_score,ef_government,ef_legal,ef_money,ef_trade,ef_regulation,ef_score,hf_score
0,2016,Albania,5.291751647,8.806809810,6.666666667,9.192592593,10.000000000,8.607142857,6.250000000,7.596280576,7.935563809,5.071813726,9.553657390,8.214899510,6.906901139,7.54,7.568140288
1,2016,Algeria,3.819566026,8.043882199,5.000000000,4.944814815,5.000000000,7.380952381,0.833333333,5.281772109,3.616782254,4.690742835,7.253893581,4.127024824,5.268992397,4.99,5.135886055
2,2016,Angola,3.451813885,8.297864524,5.000000000,8.786666667,4.000000000,6.452380952,7.500000000,6.111324364,6.496684826,2.963635116,5.606605364,5.264754044,5.518500441,5.17,5.640662182
3,2016,Argentina,5.744791027,9.040383237,10.000000000,7.795925926,7.500000000,8.738095238,10.000000000,8.099695682,5.346056135,3.904030401,5.614335974,3.982546906,5.369018774,4.84,6.469847841
4,2016,Armenia,5.003205353,8.830548737,6.666666667,6.222222222,6.500000000,7.154761905,8.000000000,6.912803602,7.003309536,5.809214965,9.521939951,8.143241004,7.378068688,7.57,7.241401801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1452,2008,Uruguay,6.400000000,9.120614313,10.000000000,9.657777778,10.000000000,9.297619048,10.000000000,8.775693261,7.315649543,5.623890096,8.447859234,8.055019795,6.162140043,7.12,7.947846630
1453,2008,Venezuela,3.100000000,6.665877012,8.333333333,7.848148148,5.583333333,6.778079218,10.000000000,6.295758656,4.842725196,2.705464235,5.296955522,3.611472215,4.666935504,4.22,5.257879328
1454,2008,Vietnam,5.600000000,9.498746366,5.000000000,6.409259259,2.750000000,4.598000988,10.000000000,6.650412616,7.400000000,6.073716608,5.253293702,5.906879173,6.307577197,6.19,6.420206308
1456,2008,Zambia,4.400000000,8.181583610,8.333333333,7.714814815,4.250000000,7.202380952,2.500000000,6.145448813,7.867647059,5.810978462,8.367633930,6.983402160,6.722174633,7.15,6.647724406


In [13]:
# Remove duplicates from Human Freedom Index dataframe
new_human_freedom_index_df = new_human_freedom_index_df.drop_duplicates()
new_human_freedom_index_df


,year,countries,pf_rol,pf_ss,pf_movement,pf_religion,pf_association,pf_expression,pf_identity,pf_score,ef_government,ef_legal,ef_money,ef_trade,ef_regulation,ef_score,hf_score
0,2016,Albania,5.291751647,8.806809810,6.666666667,9.192592593,10.000000000,8.607142857,6.250000000,7.596280576,7.935563809,5.071813726,9.553657390,8.214899510,6.906901139,7.54,7.568140288
1,2016,Algeria,3.819566026,8.043882199,5.000000000,4.944814815,5.000000000,7.380952381,0.833333333,5.281772109,3.616782254,4.690742835,7.253893581,4.127024824,5.268992397,4.99,5.135886055
2,2016,Angola,3.451813885,8.297864524,5.000000000,8.786666667,4.000000000,6.452380952,7.500000000,6.111324364,6.496684826,2.963635116,5.606605364,5.264754044,5.518500441,5.17,5.640662182
3,2016,Argentina,5.744791027,9.040383237,10.000000000,7.795925926,7.500000000,8.738095238,10.000000000,8.099695682,5.346056135,3.904030401,5.614335974,3.982546906,5.369018774,4.84,6.469847841
4,2016,Armenia,5.003205353,8.830548737,6.666666667,6.222222222,6.500000000,7.154761905,8.000000000,6.912803602,7.003309536,5.809214965,9.521939951,8.143241004,7.378068688,7.57,7.241401801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1452,2008,Uruguay,6.400000000,9.120614313,10.000000000,9.657777778,10.000000000,9.297619048,10.000000000,8.775693261,7.315649543,5.623890096,8.447859234,8.055019795,6.162140043,7.12,7.947846630
1453,2008,Venezuela,3.100000000,6.665877012,8.333333333,7.848148148,5.583333333,6.778079218,10.000000000,6.295758656,4.842725196,2.705464235,5.296955522,3.611472215,4.666935504,4.22,5.257879328
1454,2008,Vietnam,5.600000000,9.498746366,5.000000000,6.409259259,2.750000000,4.598000988,10.000000000,6.650412616,7.400000000,6.073716608,5.253293702,5.906879173,6.307577197,6.19,6.420206308
1456,2008,Zambia,4.400000000,8.181583610,8.333333333,7.714814815,4.250000000,7.202380952,2.500000000,6.145448813,7.867647059,5.810978462,8.367633930,6.983402160,6.722174633,7.15,6.647724406


In [14]:
#merge _happiness & freedom
merge_happiness_freedom=pd.merge(new_human_freedom_index_df, Happiness_Report_2015_2016_df, left_on=["countries", "year"], right_on=["country", "year"] )
merge_happiness_freedom

,year,countries,pf_rol,pf_ss,pf_movement,pf_religion,pf_association,pf_expression,pf_identity,pf_score,...,region,happiness_rank,happiness_score,economy_gdp_per_capita,family,health_life_expectancy,freedom,trust_government_corruption,generosity,dystopia_residual
0,2016,Albania,5.291751647,8.806809810,6.666666667,9.192592593,10.000000000,8.607142857,6.250000000,7.596280576,...,Central and Eastern Europe,109,4.655,0.95530,0.50163,0.73007,0.31866,0.05301,0.16840,1.92816
1,2016,Algeria,3.819566026,8.043882199,5.000000000,4.944814815,5.000000000,7.380952381,0.833333333,5.281772109,...,Middle East and Northern Africa,38,6.355,1.05266,0.83309,0.61804,0.21006,0.16157,0.07044,3.40904
2,2016,Angola,3.451813885,8.297864524,5.000000000,8.786666667,4.000000000,6.452380952,7.500000000,6.111324364,...,Sub-Saharan Africa,141,3.866,0.84731,0.66366,0.04991,0.00589,0.08434,0.12071,2.09459
3,2016,Argentina,5.744791027,9.040383237,10.000000000,7.795925926,7.500000000,8.738095238,10.000000000,8.099695682,...,Latin America and Caribbean,26,6.650,1.15137,1.06612,0.69711,0.42284,0.07296,0.10989,3.12985
4,2016,Armenia,5.003205353,8.830548737,6.666666667,6.222222222,6.500000000,7.154761905,8.000000000,6.912803602,...,Central and Eastern Europe,121,4.360,0.86086,0.62477,0.64083,0.14037,0.03616,0.07793,1.97864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,2015,United States,7.132165883,9.328352893,10.000000000,8.606296296,10.000000000,9.268106616,9.250000000,8.827569985,...,North America,15,7.119,1.39451,1.24711,0.86179,0.54604,0.15890,0.40105,2.51011
245,2015,Uruguay,6.965991287,8.305989697,10.000000000,8.905555556,7.500000000,9.321428571,10.000000000,8.390693659,...,Latin America and Caribbean,32,6.485,1.06166,1.20890,0.81160,0.60362,0.24558,0.23240,2.32142
246,2015,Venezuela,1.816299622,6.332620057,8.333333333,7.338888889,6.500000000,5.190476190,7.500000000,5.523499761,...,Latin America and Caribbean,23,6.810,1.04424,1.25596,0.72052,0.42908,0.11069,0.05841,3.19131
247,2015,Vietnam,5.165162475,8.630880895,3.333333333,5.703703704,1.500000000,5.872682796,9.250000000,6.014982826,...,Southeastern Asia,75,5.360,0.63216,0.91226,0.74676,0.59444,0.10441,0.16860,2.20173


In [15]:
merge_happiness_freedom["year"].value_counts()

2016    126
2015    123
Name: year, dtype: int64

In [16]:
#Connect Python to Postgres
import psycopg2
engine = psycopg2.connect(
    database="postgres",
    user="postgres",
    password=AWS_Password.password,
    host="happiness.c45odizugug2.us-east-2.rds.amazonaws.com",
    port='5432'
)
cursor = engine.cursor()
try:
    cursor.execute("""SELECT *
FROM public."Life_Expectancy";""")
except (Exception, psycopg2.DatabaseError) as error:
    print("Error: %s" % error)
    cursor.close()
    
# Naturally we get a list of tuples
tuples = cursor.fetchall()
colnames = [desc[0] for desc in cursor.description]
cursor.close()
# We just need to turn it into a pandas dataframe
Life_Expectancy_df = pd.DataFrame(tuples, columns=colnames)

In [17]:
Life_Expectancy_df

,Country_Name,Country_Code,Indicator_Name,Indicator_Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Aruba,ABW,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,65.66200000,66.07400000,66.44400000,66.78700000,67.11300000,67.43500000,...,75.15800000,75.29900000,75.44100000,75.58300000,75.72500000,75.86800000,76.01000000,76.15200000,None,None
1,Afghanistan,AFG,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,32.44600000,32.96200000,33.47100000,33.97100000,34.46300000,34.94800000,...,61.55300000,62.05400000,62.52500000,62.96600000,63.37700000,63.76300000,64.13000000,64.48600000,None,None
2,Angola,AGO,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,37.52400000,37.81100000,38.11300000,38.43000000,38.76000000,39.10200000,...,56.33000000,57.23600000,58.05400000,58.77600000,59.39800000,59.92500000,60.37900000,60.78200000,None,None
3,Albania,ALB,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,62.28300000,63.30100000,64.19000000,64.91400000,65.46300000,65.85000000,...,76.91400000,77.25200000,77.55400000,77.81300000,78.02500000,78.19400000,78.33300000,78.45800000,None,None
4,Andorra,AND,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Kosovo,XKX,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,None,None,None,None,None,None,...,70.14878049,70.49756098,70.79756098,71.09756098,71.34634146,71.64634146,71.94634146,72.19512195,None,None
260,"Yemen, Rep.",YEM,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,29.91900000,30.16300000,30.50000000,30.94300000,31.50100000,32.17500000,...,65.76800000,65.92000000,66.01600000,66.06600000,66.08500000,66.08700000,66.08600000,66.09600000,None,None
261,South Africa,ZAF,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,48.40600000,48.77700000,49.14200000,49.50900000,49.88800000,50.28400000,...,58.89500000,60.06000000,61.09900000,61.96800000,62.64900000,63.15300000,63.53800000,63.85700000,None,None
262,Zambia,ZMB,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,46.68700000,47.08400000,47.44600000,47.77200000,48.06800000,48.35100000,...,57.12600000,58.50200000,59.74600000,60.83100000,61.73700000,62.46400000,63.04300000,63.51000000,None,None


In [18]:
# create a new Life_expectancy table
df = pd.DataFrame(columns=["Country_Name", "Life_Expectancy", "year"])
for year in range (2015, 2017):
    lifeexp_df = Life_Expectancy_df[["Country_Name", str(year)]]
    lifeexp_df=  lifeexp_df.dropna()
    lifeexp_df=  lifeexp_df.rename(columns={str(year): "Life_Expectancy"})
    lifeexp_df['year'] = year
    df=pd.concat([df, lifeexp_df])
    
df


,Country_Name,Life_Expectancy,year
0,Aruba,75.72500000,2015
1,Afghanistan,63.37700000,2015
2,Angola,59.39800000,2015
3,Albania,78.02500000,2015
5,Arab World,71.24957141,2015
...,...,...,...
259,Kosovo,71.64634146,2016
260,"Yemen, Rep.",66.08700000,2016
261,South Africa,63.15300000,2016
262,Zambia,62.46400000,2016


In [19]:
new_life_expectancy_df=df
new_life_expectancy_df

,Country_Name,Life_Expectancy,year
0,Aruba,75.72500000,2015
1,Afghanistan,63.37700000,2015
2,Angola,59.39800000,2015
3,Albania,78.02500000,2015
5,Arab World,71.24957141,2015
...,...,...,...
259,Kosovo,71.64634146,2016
260,"Yemen, Rep.",66.08700000,2016
261,South Africa,63.15300000,2016
262,Zambia,62.46400000,2016


In [20]:
new_life_expectancy_df.isnull().sum(axis=1)

0      0
1      0
2      0
3      0
5      0
      ..
259    0
260    0
261    0
262    0
263    0
Length: 494, dtype: int64

In [21]:
# Remove NaNs from Human Freedom Index dataframe
new_life_expectancy_df=new_life_expectancy_df.dropna()
new_life_expectancy_df


,Country_Name,Life_Expectancy,year
0,Aruba,75.72500000,2015
1,Afghanistan,63.37700000,2015
2,Angola,59.39800000,2015
3,Albania,78.02500000,2015
5,Arab World,71.24957141,2015
...,...,...,...
259,Kosovo,71.64634146,2016
260,"Yemen, Rep.",66.08700000,2016
261,South Africa,63.15300000,2016
262,Zambia,62.46400000,2016


In [22]:
new_life_expectancy_df=new_life_expectancy_df.drop_duplicates()
new_life_expectancy_df


,Country_Name,Life_Expectancy,year
0,Aruba,75.72500000,2015
1,Afghanistan,63.37700000,2015
2,Angola,59.39800000,2015
3,Albania,78.02500000,2015
5,Arab World,71.24957141,2015
...,...,...,...
259,Kosovo,71.64634146,2016
260,"Yemen, Rep.",66.08700000,2016
261,South Africa,63.15300000,2016
262,Zambia,62.46400000,2016


In [23]:
final_merge = pd.merge(new_life_expectancy_df,merge_happiness_freedom, left_on=["Country_Name", "year"], right_on=["countries", "year"] )
final_merge

,Country_Name,Life_Expectancy,year,countries,pf_rol,pf_ss,pf_movement,pf_religion,pf_association,pf_expression,...,region,happiness_rank,happiness_score,economy_gdp_per_capita,family,health_life_expectancy,freedom,trust_government_corruption,generosity,dystopia_residual
0,Angola,59.39800000,2015,Angola,3.512106161,8.297864524,5.000000000,8.757037037,4.000000000,6.547619048,...,Sub-Saharan Africa,137,4.033,0.75778,0.86040,0.16683,0.10384,0.07122,0.12344,1.94939
1,Albania,78.02500000,2015,Albania,5.164786073,8.865642363,6.666666667,9.235185185,10.000000000,8.642857143,...,Central and Eastern Europe,95,4.959,0.87867,0.80434,0.81325,0.35733,0.06413,0.14272,1.89894
2,United Arab Emirates,77.28500000,2015,United Arab Emirates,6.333748881,8.023720505,3.333333333,5.124074074,0.500000000,6.153469156,...,Middle East and Northern Africa,20,6.901,1.42727,1.12575,0.80925,0.64157,0.38583,0.26428,2.24743
3,Argentina,76.06800000,2015,Argentina,5.535099829,8.962107772,10.000000000,7.888888889,7.500000000,8.619047619,...,Latin America and Caribbean,30,6.574,1.05351,1.24823,0.78723,0.44974,0.08484,0.11451,2.83600
4,Armenia,74.46700000,2015,Armenia,4.679989714,8.930382953,6.666666667,6.370370370,6.500000000,7.154761905,...,Central and Eastern Europe,127,4.350,0.76821,0.77711,0.72990,0.19847,0.03900,0.07855,1.75873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,Uruguay,77.49800000,2016,Uruguay,6.641640406,8.408217822,10.000000000,8.479629630,7.500000000,9.357142857,...,Latin America and Caribbean,29,6.545,1.18157,1.03143,0.72183,0.54388,0.21394,0.18056,2.67139
230,United States,78.53902439,2016,United States,6.949911658,9.101098169,10.000000000,8.854074074,10.000000000,9.241503522,...,North America,13,7.104,1.50796,1.04782,0.77900,0.48163,0.14868,0.41077,2.72782
231,Vietnam,75.17200000,2016,Vietnam,4.912071090,8.630880895,3.333333333,5.895925926,1.500000000,5.843436781,...,Southeastern Asia,96,5.061,0.74037,0.79117,0.66157,0.55954,0.11556,0.25075,1.94180
232,South Africa,63.15300000,2016,South Africa,5.728137568,6.044606200,10.000000000,9.364814815,10.000000000,8.904761905,...,Sub-Saharan Africa,116,4.459,1.02416,0.96053,0.18611,0.42483,0.08415,0.13656,1.64227


In [24]:
#Compare the countries columns
final_merge[["Country_Name", "countries", "country"]]

,Country_Name,countries,country
0,Angola,Angola,Angola
1,Albania,Albania,Albania
2,United Arab Emirates,United Arab Emirates,United Arab Emirates
3,Argentina,Argentina,Argentina
4,Armenia,Armenia,Armenia
...,...,...,...
229,Uruguay,Uruguay,Uruguay
230,United States,United States,United States
231,Vietnam,Vietnam,Vietnam
232,South Africa,South Africa,South Africa


In [25]:
#Drop countries and country columns
final_merge=final_merge.drop(["countries", "country"], axis=1)
final_merge

,Country_Name,Life_Expectancy,year,pf_rol,pf_ss,pf_movement,pf_religion,pf_association,pf_expression,pf_identity,...,region,happiness_rank,happiness_score,economy_gdp_per_capita,family,health_life_expectancy,freedom,trust_government_corruption,generosity,dystopia_residual
0,Angola,59.39800000,2015,3.512106161,8.297864524,5.000000000,8.757037037,4.000000000,6.547619048,7.500000000,...,Sub-Saharan Africa,137,4.033,0.75778,0.86040,0.16683,0.10384,0.07122,0.12344,1.94939
1,Albania,78.02500000,2015,5.164786073,8.865642363,6.666666667,9.235185185,10.000000000,8.642857143,6.250000000,...,Central and Eastern Europe,95,4.959,0.87867,0.80434,0.81325,0.35733,0.06413,0.14272,1.89894
2,United Arab Emirates,77.28500000,2015,6.333748881,8.023720505,3.333333333,5.124074074,0.500000000,6.153469156,0E-9,...,Middle East and Northern Africa,20,6.901,1.42727,1.12575,0.80925,0.64157,0.38583,0.26428,2.24743
3,Argentina,76.06800000,2015,5.535099829,8.962107772,10.000000000,7.888888889,7.500000000,8.619047619,10.000000000,...,Latin America and Caribbean,30,6.574,1.05351,1.24823,0.78723,0.44974,0.08484,0.11451,2.83600
4,Armenia,74.46700000,2015,4.679989714,8.930382953,6.666666667,6.370370370,6.500000000,7.154761905,8.000000000,...,Central and Eastern Europe,127,4.350,0.76821,0.77711,0.72990,0.19847,0.03900,0.07855,1.75873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,Uruguay,77.49800000,2016,6.641640406,8.408217822,10.000000000,8.479629630,7.500000000,9.357142857,10.000000000,...,Latin America and Caribbean,29,6.545,1.18157,1.03143,0.72183,0.54388,0.21394,0.18056,2.67139
230,United States,78.53902439,2016,6.949911658,9.101098169,10.000000000,8.854074074,10.000000000,9.241503522,9.250000000,...,North America,13,7.104,1.50796,1.04782,0.77900,0.48163,0.14868,0.41077,2.72782
231,Vietnam,75.17200000,2016,4.912071090,8.630880895,3.333333333,5.895925926,1.500000000,5.843436781,9.250000000,...,Southeastern Asia,96,5.061,0.74037,0.79117,0.66157,0.55954,0.11556,0.25075,1.94180
232,South Africa,63.15300000,2016,5.728137568,6.044606200,10.000000000,9.364814815,10.000000000,8.904761905,9.250000000,...,Sub-Saharan Africa,116,4.459,1.02416,0.96053,0.18611,0.42483,0.08415,0.13656,1.64227


In [26]:
final_merge.columns

Index(['Country_Name', 'Life_Expectancy', 'year', 'pf_rol', 'pf_ss',
       'pf_movement', 'pf_religion', 'pf_association', 'pf_expression',
       'pf_identity', 'pf_score', 'ef_government', 'ef_legal', 'ef_money',
       'ef_trade', 'ef_regulation', 'ef_score', 'hf_score', 'region',
       'happiness_rank', 'happiness_score', 'economy_gdp_per_capita', 'family',
       'health_life_expectancy', 'freedom', 'trust_government_corruption',
       'generosity', 'dystopia_residual'],
      dtype='object')

In [27]:
final_merge.dtypes

Country_Name                   object
Life_Expectancy                object
year                           object
pf_rol                         object
pf_ss                          object
pf_movement                    object
pf_religion                    object
pf_association                 object
pf_expression                  object
pf_identity                    object
pf_score                       object
ef_government                  object
ef_legal                       object
ef_money                       object
ef_trade                       object
ef_regulation                  object
ef_score                       object
hf_score                       object
region                         object
happiness_rank                  int64
happiness_score                object
economy_gdp_per_capita         object
family                         object
health_life_expectancy         object
freedom                        object
trust_government_corruption    object
generosity  

In [39]:
# Change the data types of everything except Happiness Rank, Country_Name, and regionto float
changed_columns = final_merge[['Life_Expectancy', 'year', 'pf_rol', 'pf_ss',
       'pf_movement', 'pf_religion', 'pf_association', 'pf_expression',
       'pf_identity', 'pf_score', 'ef_government', 'ef_legal', 'ef_money',
       'ef_trade', 'ef_regulation', 'ef_score', 'hf_score', 'happiness_score', 'economy_gdp_per_capita', 'family',
       'health_life_expectancy', 'freedom', 'trust_government_corruption',
       'generosity', 'dystopia_residual']].astype('float')
changed_columns

,Life_Expectancy,year,pf_rol,pf_ss,pf_movement,pf_religion,pf_association,pf_expression,pf_identity,pf_score,...,ef_score,hf_score,happiness_score,economy_gdp_per_capita,family,health_life_expectancy,freedom,trust_government_corruption,generosity,dystopia_residual
0,59.398000,2015.0,3.512106,8.297865,5.000000,8.757037,4.0,6.547619,7.500000,6.132958,...,5.44,5.786479,4.033,0.75778,0.86040,0.16683,0.10384,0.07122,0.12344,1.94939
1,78.025000,2015.0,5.164786,8.865642,6.666667,9.235185,10.0,8.642857,6.250000,7.587078,...,7.53,7.558539,4.959,0.87867,0.80434,0.81325,0.35733,0.06413,0.14272,1.89894
2,77.285000,2015.0,6.333749,8.023721,3.333333,5.124074,0.5,6.153469,0.000000,5.100455,...,7.45,6.275228,6.901,1.42727,1.12575,0.80925,0.64157,0.38583,0.26428,2.24743
3,76.068000,2015.0,5.535100,8.962108,10.000000,7.888889,7.5,8.619048,10.000000,8.025096,...,4.98,6.502548,6.574,1.05351,1.24823,0.78723,0.44974,0.08484,0.11451,2.83600
4,74.467000,2015.0,4.679990,8.930383,6.666667,6.370370,6.5,7.154762,8.000000,6.871773,...,7.59,7.230887,4.350,0.76821,0.77711,0.72990,0.19847,0.03900,0.07855,1.75873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,77.498000,2016.0,6.641640,8.408218,10.000000,8.479630,7.5,9.357143,10.000000,8.296142,...,7.16,7.728071,6.545,1.18157,1.03143,0.72183,0.54388,0.21394,0.18056,2.67139
230,78.539024,2016.0,6.949912,9.101098,10.000000,8.854074,10.0,9.241504,9.250000,8.747310,...,8.03,8.388655,7.104,1.50796,1.04782,0.77900,0.48163,0.14868,0.41077,2.72782
231,75.172000,2016.0,4.912071,8.630881,3.333333,5.895926,1.5,5.843437,9.250000,5.968008,...,6.42,6.194004,5.061,0.74037,0.79117,0.66157,0.55954,0.11556,0.25075,1.94180
232,63.153000,2016.0,5.728138,6.044606,10.000000,9.364815,10.0,8.904762,9.250000,7.695144,...,6.65,7.172572,4.459,1.02416,0.96053,0.18611,0.42483,0.08415,0.13656,1.64227


In [40]:
other_columns = final_merge[['year','Country_Name', 'happiness_rank', 'region']]
other_columns

,year,Country_Name,happiness_rank,region
0,2015,Angola,137,Sub-Saharan Africa
1,2015,Albania,95,Central and Eastern Europe
2,2015,United Arab Emirates,20,Middle East and Northern Africa
3,2015,Argentina,30,Latin America and Caribbean
4,2015,Armenia,127,Central and Eastern Europe
...,...,...,...,...
229,2016,Uruguay,29,Latin America and Caribbean
230,2016,United States,13,North America
231,2016,Vietnam,96,Southeastern Asia
232,2016,South Africa,116,Sub-Saharan Africa


In [41]:
# Merge changed_columns and other_columns
merged_dataset = pd.merge(changed_columns, other_columns, left_on=["year"], right_on=["year"] )
merged_dataset

,Life_Expectancy,year,pf_rol,pf_ss,pf_movement,pf_religion,pf_association,pf_expression,pf_identity,pf_score,...,economy_gdp_per_capita,family,health_life_expectancy,freedom,trust_government_corruption,generosity,dystopia_residual,Country_Name,happiness_rank,region
0,59.398,2015,3.512106,8.297865,5.0,8.757037,4.0,6.547619,7.500000,6.132958,...,0.75778,0.86040,0.16683,0.10384,0.07122,0.12344,1.94939,Angola,137,Sub-Saharan Africa
1,59.398,2015,3.512106,8.297865,5.0,8.757037,4.0,6.547619,7.500000,6.132958,...,0.75778,0.86040,0.16683,0.10384,0.07122,0.12344,1.94939,Albania,95,Central and Eastern Europe
2,59.398,2015,3.512106,8.297865,5.0,8.757037,4.0,6.547619,7.500000,6.132958,...,0.75778,0.86040,0.16683,0.10384,0.07122,0.12344,1.94939,United Arab Emirates,20,Middle East and Northern Africa
3,59.398,2015,3.512106,8.297865,5.0,8.757037,4.0,6.547619,7.500000,6.132958,...,0.75778,0.86040,0.16683,0.10384,0.07122,0.12344,1.94939,Argentina,30,Latin America and Caribbean
4,59.398,2015,3.512106,8.297865,5.0,8.757037,4.0,6.547619,7.500000,6.132958,...,0.75778,0.86040,0.16683,0.10384,0.07122,0.12344,1.94939,Armenia,127,Central and Eastern Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27375,60.294,2016,3.507599,7.610859,5.0,6.427778,2.5,6.650000,3.333333,5.170726,...,0.35041,0.71478,0.15950,0.25429,0.08582,0.18503,2.44270,Uruguay,29,Latin America and Caribbean
27376,60.294,2016,3.507599,7.610859,5.0,6.427778,2.5,6.650000,3.333333,5.170726,...,0.35041,0.71478,0.15950,0.25429,0.08582,0.18503,2.44270,United States,13,North America
27377,60.294,2016,3.507599,7.610859,5.0,6.427778,2.5,6.650000,3.333333,5.170726,...,0.35041,0.71478,0.15950,0.25429,0.08582,0.18503,2.44270,Vietnam,96,Southeastern Asia
27378,60.294,2016,3.507599,7.610859,5.0,6.427778,2.5,6.650000,3.333333,5.170726,...,0.35041,0.71478,0.15950,0.25429,0.08582,0.18503,2.44270,South Africa,116,Sub-Saharan Africa


In [44]:
final_df = pd.get_dummies(merged_dataset, columns=["Country_Name", "region"])
final_df.head()

,Life_Expectancy,year,pf_rol,pf_ss,pf_movement,pf_religion,pf_association,pf_expression,pf_identity,pf_score,...,region_Australia and New Zealand,region_Central and Eastern Europe,region_Eastern Asia,region_Latin America and Caribbean,region_Middle East and Northern Africa,region_North America,region_Southeastern Asia,region_Southern Asia,region_Sub-Saharan Africa,region_Western Europe
0,59.398,2015,3.512106,8.297865,5.0,8.757037,4.0,6.547619,7.5,6.132958,...,0,0,0,0,0,0,0,0,1,0
1,59.398,2015,3.512106,8.297865,5.0,8.757037,4.0,6.547619,7.5,6.132958,...,0,1,0,0,0,0,0,0,0,0
2,59.398,2015,3.512106,8.297865,5.0,8.757037,4.0,6.547619,7.5,6.132958,...,0,0,0,0,1,0,0,0,0,0
3,59.398,2015,3.512106,8.297865,5.0,8.757037,4.0,6.547619,7.5,6.132958,...,0,0,0,1,0,0,0,0,0,0
4,59.398,2015,3.512106,8.297865,5.0,8.757037,4.0,6.547619,7.5,6.132958,...,0,1,0,0,0,0,0,0,0,0


In [45]:
final_df.columns

Index(['Life_Expectancy', 'year', 'pf_rol', 'pf_ss', 'pf_movement',
       'pf_religion', 'pf_association', 'pf_expression', 'pf_identity',
       'pf_score',
       ...
       'region_Australia and New Zealand', 'region_Central and Eastern Europe',
       'region_Eastern Asia', 'region_Latin America and Caribbean',
       'region_Middle East and Northern Africa', 'region_North America',
       'region_Southeastern Asia', 'region_Southern Asia',
       'region_Sub-Saharan Africa', 'region_Western Europe'],
      dtype='object', length=156)

In [51]:
final_df.dtypes

Life_Expectancy              float64
year                          object
pf_rol                       float64
pf_ss                        float64
pf_movement                  float64
                              ...   
region_North America           uint8
region_Southeastern Asia       uint8
region_Southern Asia           uint8
region_Sub-Saharan Africa      uint8
region_Western Europe          uint8
Length: 156, dtype: object

In [46]:
#final_df.to_csv("Final_Dataset.csv")

# Machine Learning Model

In [47]:
#We will be using a supervised model

In [48]:
#Since we are charting the happiness score.
#we will focus on these factors
#life expectancy at birth, Freedom index to make life choices for the purposes of our analysis
#Our outputs will be how happy each country is based on the factors imputed.

In [ ]:
#We will train test split
#We will import the “train_test_split” from sklearn.model_selection.
#We will define the  x as the independent variable (this what the model takes in) and y as the dependent variable (the output we want to predict) and we will use the  75% - 25% default. Both x and y need to be split in train and test. Once you test the model, the model will take x_test data and check the predicted output/label against the y_test (which helps to determine the accuracy -> how close the model got the predicted y to the actual (test) . 


## Split the data into training and testing data

In [49]:
# Create our features
X = pd.get_dummies(final_df.drop(columns="happiness_score"))

# Create our target
y = final_df["happiness_score"]

In [50]:
# Show the resulting dataframe
X.describe()

,Life_Expectancy,pf_rol,pf_ss,pf_movement,pf_religion,pf_association,pf_expression,pf_identity,pf_score,ef_government,...,region_Eastern Asia,region_Latin America and Caribbean,region_Middle East and Northern Africa,region_North America,region_Southeastern Asia,region_Southern Asia,region_Sub-Saharan Africa,region_Western Europe,year_2015.0,year_2016.0
count,27380.000000,27380.000000,27380.000000,27380.000000,27380.000000,27380.000000,27380.000000,27380.000000,27380.000000,27380.000000,...,27380.000000,27380.000000,27380.000000,27380.000000,27380.000000,27380.000000,27380.000000,27380.000000,27380.000000,27380.000000
mean,73.385170,5.334129,8.315060,7.919771,7.503735,7.232834,7.889274,7.070806,7.173939,6.466614,...,0.025639,0.153835,0.119649,0.017093,0.068371,0.042732,0.217969,0.170928,0.491454,0.508546
std,7.497901,1.672476,1.359442,2.564217,1.474760,2.690984,1.511445,3.023229,1.437693,1.302613,...,0.158059,0.360797,0.324557,0.129620,0.252386,0.202256,0.412874,0.376453,0.499936,0.499936
min,53.112000,2.198018,3.963850,0.000000,3.214815,0.500000,1.944578,0.000000,3.116028,3.482223,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,68.977000,4.070537,7.520505,6.666667,6.447778,5.000000,7.047619,5.000000,6.092465,5.562500,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,75.052000,4.901722,8.544921,8.333333,7.920370,8.000000,8.226190,8.000000,7.059030,6.537883,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,79.646000,6.676722,9.574248,10.000000,8.610741,9.500000,9.214286,9.250000,8.521920,7.510213,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,83.984878,8.723094,9.959940,10.000000,9.877778,10.000000,9.797619,10.000000,9.407869,9.528485,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [52]:
# Check the balance of our target values
y.value_counts()

3.739    236
2.905    234
7.119    234
5.123    234
4.252    234
        ... 
6.455    116
4.642    116
3.667    116
5.791    116
6.750    116
Name: happiness_score, Length: 229, dtype: int64

In [53]:
# Perform the train, test, split of the data

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(20535, 156)

In [ ]:
#Initial Imports
#Preprocess the data
#Load the data
#Clean data
#Split data into train test split
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
                                                    
#X_train.shape
 
#X = StandardScaler().fit_transform(X)


In [ ]:
#Separate the features (X) from the target (y)
#Split data into training and testing
#Create a logistic regression model - is this what we're doing?
#Fit the model using the training data
#Make predictions
#Print accuracy and confusion matrix


In [ ]:
#Merging Happiness